In [6]:
import numpy as np
import pandas as pd
import streamlit as st
from loaders import get_datetime_cols, get_worksheet, get_worksheet_location
from connections import get_google_conn
import streamlit as st
import pandas as pd

In [7]:
sheet_url = st.secrets["private_gsheets_url"]

In [11]:
cheki_df = get_worksheet(sheet_url, 0)
person_df = get_worksheet(sheet_url, 1)
venue_df = get_worksheet_location(sheet_url, 3)

In [15]:
person_df

,entity,name1,group1,name2,group2,name3,group3,name4,group4,name5,group5
1,勝月蘭丸@1682,勝月蘭丸,1682,,,,,,,,
2,承和 薫@1682,承和 薫,1682,,,,,,,,
3,月見夜零@1682,月見夜零,1682,,,,,,,,
4,神山おだし@1682,神山おだし,1682,,,,,,,,
5,天野ひろこ@.BPM,天野ひろこ,.BPM,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
484,ちぃたん☆,ちぃたん☆,,,,,,,,,
485,上村らな@シュアネス,上村らな,シュアネス,,,,,,,,
486,ひめみや@Zero Project,ひめみや,Zero Project,,,,,,,,
487,白愛るみ@↑ANiMO↑,白愛るみ,↑ANiMO↑,,,,,,,,


In [35]:
persons_list = []
for idx, row in person_df.iterrows():
    person_id = idx
    entities = []
    for i in range(5, 0, -1):
        tmp_dict = {}
        if row[f"name{i}"]:
            tmp_dict["name"] = row[f"name{i}"]
        if row[f"group{i}"]:
            tmp_dict["group"] = row[f"group{i}"]
        if tmp_dict:
            entities.append(tmp_dict)
    persons_list.append({"person_id": idx, "entities": entities, "current": row["entity"]})


In [38]:
pd.DataFrame(persons_list)

,person_id,entities
0,1,"[{'name': '勝月蘭丸', 'group': '1682'}]"
1,2,"[{'name': '承和 薫', 'group': '1682'}]"
2,3,"[{'name': '月見夜零', 'group': '1682'}]"
3,4,"[{'name': '神山おだし', 'group': '1682'}]"
4,5,"[{'name': '天野ひろこ', 'group': '.BPM'}]"
...,...,...
483,484,[{'name': 'ちぃたん☆'}]
484,485,"[{'name': '上村らな', 'group': 'シュアネス'}]"
485,486,"[{'name': 'ひめみや', 'group': 'Zero Project'}]"
486,487,"[{'name': '白愛るみ', 'group': '↑ANiMO↑'}]"
